## <span style="color:#ff5f27">👨🏻‍🏫 Create Deployment </span>

Creating a deployment for the recommendation system.


## <span style="color:#ff5f27">📝 Imports </span>


In [2]:
import os
import hopsworks

## <span style="color:#ff5f27">🔮 Connect to Hopsworks </span>

In [3]:
import hopsworks

project = hopsworks.login()

# Connect to Hopsworks Model Registry
mr = project.get_model_registry()

dataset_api = project.get_dataset_api()

2025-06-24 17:24:29,284 INFO: Initializing external client
2025-06-24 17:24:29,288 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-06-24 17:24:30,735 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


# Testing the recsys 

In [4]:
weather_ranking_model = mr.get_best_model(
    name="weather_ranking_model", 
    metric="fscore", 
    direction="max",
)
weather_ranking_model


Model(name: 'weather_ranking_model', version: 1)

In [6]:
no_weather_ranking_model = mr.get_best_model(
    name="no_weather_ranking_model", 
    metric="fscore", 
    direction="max",
)
no_weather_ranking_model

Model(name: 'no_weather_ranking_model', version: 1)

In [7]:
#project = hopsworks.connection().get_project()
fs = project.get_feature_store()
        
events_fv = fs.get_feature_view(
            name="events", 
            version=1,
        )
# Retrieve the 'users' feature view
users_fv = fs.get_feature_view(
    name="users", 
    version=1,
)

interactions_fv = fs.get_feature_view(
            name="interactions", 
            version=1,
        )

print(events_fv)

In [8]:
# Retrieve the 'candidate_embeddings' feature view
candidate_index = fs.get_feature_view(
    name="candidate_embeddings", 
    version=1,
)

In [9]:
event_features = [feat.name for feat in events_fv.schema]


In [10]:

mr = project.get_model_registry()
model = mr.get_model(
    name="weather_ranking_model", 
    version=1,
)

# Extract input schema from the model
input_schema = model.model_schema["input_schema"]["columnar_schema"]
input_schema

Downloading: 0.000%|          | 0/1283 elapsed<00:00 remaining<?

[{'name': 'interaction_distance_to_event', 'type': 'float64'},
 {'name': 'event_type', 'type': 'object'},
 {'name': 'event_city', 'type': 'object'},
 {'name': 'weather_condition', 'type': 'object'},
 {'name': 'temperature', 'type': 'float64'},
 {'name': 'attendance_rate', 'type': 'float64'},
 {'name': 'event_indoor_capability', 'type': 'bool'},
 {'name': 'user_city', 'type': 'object'},
 {'name': 'indoor_outdoor_preference', 'type': 'object'},
 {'name': 'age', 'type': 'int64'},
 {'name': 'user_interests', 'type': 'object'},
 {'name': 'user_weather_condition', 'type': 'object'},
 {'name': 'user_temperature', 'type': 'float64'},
 {'name': 'user_precipitation', 'type': 'float64'}]

In [11]:
weather_ranking_model_feature_names = [feat["name"] for feat in input_schema]
weather_ranking_model_feature_names

['interaction_distance_to_event',
 'event_type',
 'event_city',
 'weather_condition',
 'temperature',
 'attendance_rate',
 'event_indoor_capability',
 'user_city',
 'indoor_outdoor_preference',
 'age',
 'user_interests',
 'user_weather_condition',
 'user_temperature',
 'user_precipitation']

In [13]:
import pandas as pd
import numpy as np
import joblib
import os
from catboost import Pool

inputs = {
  "instances": [
    [
      {
        "user_id": "OX425G",
        "query_emb": [0.214135289, 0.571055949, 0.330709577, -0.225899458, -0.308674961, 
        -0.0115124583, 0.0730511621, -0.495835781, 0.625569344, -0.0438038409, 
        0.263472944, -0.58485353, -0.307070434, 0.0414443575, -0.321789205, 
        0.966559, 0.127463, -0.392714, 0.845132, -0.512387, 0.253901, 
        -0.764589, 0.431267, 0.087342, -0.629045, 0.318976, -0.146782, 
        0.573921, -0.087625, 0.934261, -0.271843, 0.652197]
      }
    ]
  ]
}

# Extract the input instance
inputs = inputs["instances"][0][0]

# Extract customer_id from inputs
user_id = inputs["user_id"]

# Search for candidate items
neighbors = candidate_index.find_neighbors(
    inputs["query_emb"], 
    k=10,
)
neighbors = [neighbor[0] for neighbor in neighbors]
print(neighbors)

event_id_list = [
    event_id
    for event_id 
    in neighbors 
]
event_id_df = pd.DataFrame({"event_id" : event_id_list})
print("creating event data...")

events_df = (
    events_fv.get_batch_data(
        # start_time="20250101",
        # end_time="20250624",
        primary_key=True,
        event_time=True,
        read_options={"use_arrow_flight": True}
    )
)
events_data_df = events_df[events_df["event_id"].isin(event_id_list)]

# Add customer features
user_data_df = (
    users_fv.get_batch_data(
        primary_key=True,
        read_options={"use_arrow_flight": True}
    )
)

# Add interactions features
interaction_data_df = (
    interactions_fv.get_batch_data(
        primary_key=True,
        read_options={"use_arrow_flight": True}
    )
)

# get ranking features
ranking_model_inputs = events_data_df.merge(interaction_data_df, on="event_id").\
  merge(user_data_df, on="user_id")

# Select only the features required by the ranking model
#ranking_model_inputs = ranking_model_inputs[weather_ranking_model_feature_names]
# display(ranking_model_inputs.head()) 
inputs = { 
            "inputs" : [{"ranking_features": ranking_model_inputs.values.tolist(), "event_ids": event_id_list}]
        }
print(inputs)


# Load the pre-trained model
os.environ['ARTIFACT_FILES_PATH'] = '/home/nkama/masters_thesis_project/thesis/models'

model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/weather_ranking_model.pkl")

model_feature_names = model.feature_names_

# Align prediction data columns with model's feature names
ranking_model_inputs = ranking_model_inputs[model_feature_names]

# and 'event_id_list' contains the corresponding event IDs
categorical_features = ranking_model_inputs.select_dtypes(include=["object", "bool"]).columns.tolist()

pool_features = Pool(data=ranking_model_inputs, cat_features=categorical_features)

# Predict probabilities for the positive class
probabilities = model.predict_proba(pool_features)

# Extract the probability of the positive class
positive_class_index = list(model.classes_).index(1)
scores = probabilities[:, positive_class_index]

# Pair each score with its corresponding event ID
ranking = list(zip(scores, event_id_list))

# Sort the ranking in descending order of scores
ranking.sort(key=lambda x: x[0], reverse=True)

# Prepare the final output
rank = {
    "ranking": ranking
}

print(rank)


['FS955S', 'CM824R', 'CR399H', 'TW818N', 'VF065U', 'OH995L', 'PA960E', 'QX017G', 'TO649I', 'YL323F']
creating event data...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.69s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.92s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.48s) 
{'inputs': [{'ranking_features': [['FS955S', 'Synchronized client-driven workforce Sports & Fitness in Dubai', 'Sports & Fitness', 25.01075164544951, 55.54634522856133, 'Dubai', '2025-06-01 10:01:27.913269', 360, 'Clear', 34.7, 0.0, 2.9563114249415383, False, 'LU170D', 'WA283W', 'maybe', '2025-05-24 02:51:10.222569', 60.26155097929416, 1, 25.52448963074197, 55.74306931194354, 'Dubai', 'outdoor', 46, 'art,travel,fashion', '2024-04-26 16:26:39.347184', 13, 'Rain', 31.5, 18.94], ['CM824R', 'Up-sized regional frame Sports & Fitness in Dubai', 'Sports & Fitness', 25.39250480071952, 55.10723236055948, 'Dubai', '2025-

## <span style="color:#ff5f27">🚀 Ranking Model Deployment </span>



Since the ranking model is a CatBoost model we need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it.


### Weather model

In [111]:
%%writefile weather_ranking_transformer.py
# weather_ranking_transformer.py
import os
import pandas as pd
import hopsworks
from datetime import datetime

class Transformer:

    def __init__(self):
        project = hopsworks.connection().get_project()
        self.fs = project.get_feature_store()

        # Feature Views
        self.events_fv       = self.fs.get_feature_view(name="events",               version=1)
        self.users_fv        = self.fs.get_feature_view(name="users",                version=1)
        self.interactions_fv = self.fs.get_feature_view(name="interactions",         version=1)
        self.candidate_index = self.fs.get_feature_view(name="candidate_embeddings", version=1)

        # Model schema
        mr = project.get_model_registry()
        model = mr.get_best_model(
            name="weather_ranking_model",
            metric="fscore",
            direction="max"
        )
        schema = model.model_schema["input_schema"]["columnar_schema"]
        self.feature_names = [f["name"] for f in schema]

    def preprocess(self, inputs):
        # Unpack two nested lists to get the payload dict
        inst      = inputs["instances"][0][0]
        query_emb = inst["query_emb"]

        # 1) k-NN retrieval of candidate event IDs
        neighbors = self.candidate_index.find_neighbors(query_emb, k=100)
        event_ids = [n[0] for n in neighbors]

        # 2) Batch fetch, parse times, and filter out past events
        events_df = self.events_fv.get_batch_data(
            primary_key=True,
            event_time=True,
            read_options={"use_arrow_flight": True}
        )
        events_df["start_time"] = pd.to_datetime(events_df["start_time"])
        now       = pd.Timestamp(datetime.utcnow())
        future    = events_df[events_df["start_time"] >= now]
        candidates_df = future[future["event_id"].isin(event_ids)]

        # 3) Batch fetch user & interaction features
        user_df = self.users_fv.get_batch_data(
            primary_key=True,
            read_options={"use_arrow_flight": True}
        )
        int_df  = self.interactions_fv.get_batch_data(
            primary_key=True,
            read_options={"use_arrow_flight": True}
        )

        # 4) Assemble DataFrame
        rank_df = (
            candidates_df
            .merge(int_df, on="event_id")
            .merge(user_df, on="user_id")
        )[self.feature_names]

        # 5) Return under "inputs" for the Predictor
        return {
            "inputs": [{
                "ranking_features": rank_df.values.tolist(),
                "event_ids":        candidates_df["event_id"].tolist()
            }]
        }

    def postprocess(self, outputs):
        # outputs is the Predictor’s return value
        return outputs




Overwriting weather_ranking_transformer.py


In [112]:
%%writefile weather_ranking_predictor.py

# weather_ranking_predictor.py
import os
import joblib
import pandas as pd
from catboost import Pool

class Predict:

    def __init__(self):
        model_dir = os.environ["MODEL_FILES_PATH"]
        self.model = joblib.load(os.path.join(model_dir, "weather_ranking_model.pkl"))
        self.feature_names = self.model.feature_names_

    def predict(self, inputs):
        # --- Fix is here: unpack from inputs["inputs"][0], not inputs[0] ---
        batch     = inputs["inputs"][0]
        features  = batch["ranking_features"]
        event_ids = batch["event_ids"]

        # Construct DataFrame with correct columns
        df = pd.DataFrame(features, columns=self.feature_names)

        # Cast categorical columns to string
        cat_cols = df.select_dtypes(include=["object", "bool"]).columns.tolist()
        for c in cat_cols:
            df[c] = df[c].astype(str)

        # Create Pool & score
        pool  = Pool(data=df, cat_features=cat_cols)
        probs = self.model.predict_proba(pool)
        pos_idx = list(self.model.classes_).index(1)
        scores  = probs[:, pos_idx].tolist()

        # Return final outputs
        return {
            "scores":    scores,
            "event_ids": event_ids
        }





Overwriting weather_ranking_predictor.py


In [113]:
# deploy_event_recsys.py
import os
import hopsworks
from hsml.transformer import Transformer

# Login to Hopsworks
project = hopsworks.login()
dataset_api = project.get_dataset_api()
mr = project.get_model_registry()

# === Upload Scripts to Hopsworks ===

# Upload transformer script
uploaded_transformer_path = dataset_api.upload(
    "weather_ranking_transformer.py",
    "Resources",
    overwrite=True,
)

# Upload predictor script
uploaded_predictor_path = dataset_api.upload(
    "weather_ranking_predictor.py",
    "Resources",
    overwrite=True,
)

# Build absolute script paths for deployment
transformer_script_path = os.path.join("/Projects", project.name, uploaded_transformer_path)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_predictor_path)

# === Retrieve Ranking Model ===
ranking_model = mr.get_best_model(
    name="weather_ranking_model",
    metric="fscore",
    direction="max"
)

# === Define Transformer ===
ranking_transformer = Transformer(
    script_file=transformer_script_path,
    resources={"num_instances": 0},
)

# === Deploy Model ===
ranking_deployment_name = "weatherrankingdeployment"
weather_ranking_deployment = ranking_model.deploy(
    name=ranking_deployment_name,
    description="Deployment that retrieves and scores candidate events for a user",
    script_file=predictor_script_path,
    resources={"num_instances": 0},
    transformer=ranking_transformer,
)


2025-06-25 18:20:25,304 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-06-25 18:20:25,319 INFO: Initializing external client
2025-06-25 18:20:25,322 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-06-25 18:20:26,683 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


Uploading /home/nkama/masters_thesis_project/thesis/notebooks/weather_ranking_transformer.py: 0.000%|         …

Uploading /home/nkama/masters_thesis_project/thesis/notebooks/weather_ranking_predictor.py: 0.000%|          |…

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/1220788/deployments/377886
Before making predictions, start the deployment by using `.start()`


In [114]:
# Start the deployment
weather_ranking_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [ ]:
# Test the Deployment 
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates['ranking'][:k]]

# Example input to test the deployment
test_input = {
    "instances": [[{
        "user_id": "OX425G",
        "query_emb": [0.214135289, 0.571055949, 0.330709577, -0.225899458,
                       -0.308674961, -0.0115124583, 0.0730511621, -0.495835781,
                       0.625569344, -0.0438038409, 0.263472944, -0.58485353,
                       -0.307070434, 0.0414443575, -0.321789205, 0.966559,
                       0.127463, -0.392714, 0.845132, -0.512387, 0.253901,
                       -0.764589, 0.431267, 0.087342, -0.629045, 0.318976,
                       -0.146782, 0.573921, -0.087625, 0.934261, -0.271843, 0.652197]
    }]]
}

# Predict
ranked_output = weather_ranking_deployment.predict(test_input)
recommendations = get_top_recommendations(ranked_output, k=3)

print("Top recommended events:", recommendations)


In [ ]:

#ranking_deployment.get_logs(component="predictor", tail=100)
ranking_deployment.get_logs(component="transformer", tail=100)


In [ ]:
import os


# Set the environment variable
os.environ['ARTIFACT_FILES_PATH'] = '/home/nkama/masters_thesis_project/thesis/models'

# Optional: Verify that the variable is set
print(os.environ.get('ARTIFACT_FILES_PATH'))

model_path = os.path.join(os.environ["ARTIFACT_FILES_PATH"], "weather_ranking_model.pkl")
print(os.path.exists(model_path))  # Should return True



### No-Weather Model

In [ ]:
%%writefile no_weather_ranking_transformer.py
# weather_ranking_transformer.py
import os
import pandas as pd
import hopsworks
from datetime import datetime

class Transformer:

    def __init__(self):
        project = hopsworks.connection().get_project()
        self.fs = project.get_feature_store()

        # Feature Views
        self.events_fv       = self.fs.get_feature_view(name="events",               version=1)
        self.users_fv        = self.fs.get_feature_view(name="users",                version=1)
        self.interactions_fv = self.fs.get_feature_view(name="interactions",         version=1)
        self.candidate_index = self.fs.get_feature_view(name="candidate_embeddings", version=1)

        # Model schema
        mr = project.get_model_registry()
        model = mr.get_best_model(
            name="no_weather_ranking_model",
            metric="fscore",
            direction="max"
        )
        schema = model.model_schema["input_schema"]["columnar_schema"]
        self.feature_names = [f["name"] for f in schema]

    def preprocess(self, inputs):
        # Unpack two nested lists to get the payload dict
        inst      = inputs["instances"][0][0]
        query_emb = inst["query_emb"]

        # 1) k-NN retrieval of candidate event IDs
        neighbors = self.candidate_index.find_neighbors(query_emb, k=100)
        event_ids = [n[0] for n in neighbors]

        # 2) Batch fetch, parse times, and filter out past events
        events_df = self.events_fv.get_batch_data(
            primary_key=True,
            event_time=True,
            read_options={"use_arrow_flight": True}
        )
        events_df["start_time"] = pd.to_datetime(events_df["start_time"])
        now       = pd.Timestamp(datetime.utcnow())
        future    = events_df[events_df["start_time"] >= now]
        candidates_df = future[future["event_id"].isin(event_ids)]

        # 3) Batch fetch user & interaction features
        user_df = self.users_fv.get_batch_data(
            primary_key=True,
            read_options={"use_arrow_flight": True}
        )
        int_df  = self.interactions_fv.get_batch_data(
            primary_key=True,
            read_options={"use_arrow_flight": True}
        )

        # 4) Assemble DataFrame
        rank_df = (
            candidates_df
            .merge(int_df, on="event_id")
            .merge(user_df, on="user_id")
        )[self.feature_names]

        # 5) Return under "inputs" for the Predictor
        return {
            "inputs": [{
                "ranking_features": rank_df.values.tolist(),
                "event_ids":        candidates_df["event_id"].tolist()
            }]
        }

    def postprocess(self, outputs):
        # outputs is the Predictor’s return value
        return outputs




In [ ]:
%%writefile no_weather_ranking_predictor.py

# weather_ranking_predictor.py
import os
import joblib
import pandas as pd
from catboost import Pool

class Predict:

    def __init__(self):
        model_dir = os.environ["MODEL_FILES_PATH"]
        self.model = joblib.load(os.path.join(model_dir, "no_weather_ranking_model.pkl"))
        self.feature_names = self.model.feature_names_

    def predict(self, inputs):
        # --- Fix is here: unpack from inputs["inputs"][0], not inputs[0] ---
        batch     = inputs["inputs"][0]
        features  = batch["ranking_features"]
        event_ids = batch["event_ids"]

        # Construct DataFrame with correct columns
        df = pd.DataFrame(features, columns=self.feature_names)

        # Cast categorical columns to string
        cat_cols = df.select_dtypes(include=["object", "bool"]).columns.tolist()
        for c in cat_cols:
            df[c] = df[c].astype(str)

        # Create Pool & score
        pool  = Pool(data=df, cat_features=cat_cols)
        probs = self.model.predict_proba(pool)
        pos_idx = list(self.model.classes_).index(1)
        scores  = probs[:, pos_idx].tolist()

        # Return final outputs
        return {
            "scores":    scores,
            "event_ids": event_ids
        }


In [ ]:
# deploy no-weather event recsys
import os
import hopsworks
from hsml.transformer import Transformer

# Login to Hopsworks
project = hopsworks.login()
dataset_api = project.get_dataset_api()
mr = project.get_model_registry()


# Upload transformer script
uploaded_transformer_path = dataset_api.upload(
    "no_weather_ranking_transformer.py",
    "Resources",
    overwrite=True,
)

# Upload predictor script
uploaded_predictor_path = dataset_api.upload(
    "no_weather_ranking_predictor.py",
    "Resources",
    overwrite=True,
)

# Build absolute script paths for deployment
transformer_script_path = os.path.join("/Projects", project.name, uploaded_transformer_path)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_predictor_path)

# Retrieve Ranking Model 
ranking_model = mr.get_best_model(
    name="no_weather_ranking_model",
    metric="fscore",
    direction="max"
)

# Define Transformer 
ranking_transformer = Transformer(
    script_file=transformer_script_path,
    resources={"num_instances": 0},
)

# Deploy Model
ranking_deployment_name = "noweatherrankingdeployment"
no_weather_ranking_deployment = ranking_model.deploy(
    name=ranking_deployment_name,
    description="Deployment that retrieves and scores candidate events for a user",
    script_file=predictor_script_path,
    resources={"num_instances": 0},
    transformer=ranking_transformer,
)


In [ ]:
# Start the deployment
no_weather_ranking_deployment.start()


In [ ]:
# Test the Deployment
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates['ranking'][:k]]

# Example input to test the deployment
test_input = {
    "instances": [[{
        "user_id": "OX425G",
        "query_emb": [0.214135289, 0.571055949, 0.330709577, -0.225899458,
                       -0.308674961, -0.0115124583, 0.0730511621, -0.495835781,
                       0.625569344, -0.0438038409, 0.263472944, -0.58485353,
                       -0.307070434, 0.0414443575, -0.321789205, 0.966559,
                       0.127463, -0.392714, 0.845132, -0.512387, 0.253901,
                       -0.764589, 0.431267, 0.087342, -0.629045, 0.318976,
                       -0.146782, 0.573921, -0.087625, 0.934261, -0.271843, 0.652197]
    }]]
}

# Predict
ranked_output = no_weather_ranking_deployment.predict(test_input)
recommendations = get_top_recommendations(ranked_output, k=3)

print("Top recommended events:", recommendations)


# Query model Deployment

In [59]:
# Retrieve the 'query_model' from the Model Registry
query_model = mr.get_model(
    name="query_model",
    version=1,
)

In [146]:
%%writefile querymodel_transformer.py

import os
import numpy as np
import pandas as pd
from datetime import datetime

import hopsworks

import logging


class Transformer(object):
    
    def __init__(self): 
        # Connect to the Hopsworks
        project = hopsworks.connection().get_project()
        ms = project.get_model_serving()
        
        # Retrieve the 'users' feature view
        fs = project.get_feature_store()
        self.users_fv = fs.get_feature_view(
            name="users", 
            version=1,
        )
        # Retrieve the ranking deployment 
        self.ranking_server = ms.get_deployment("weatherrankingdeployment")
        
    
    def preprocess(self, inputs):
        # Check if the input data contains a key named "instances"
        # and extract the actual data if present
        inputs = inputs["instances"] if "instances" in inputs else inputs

        # Extract user_id from the inputs
        user_id = inputs["user_id"]

        # Get user features
        user_features = self.users_fv.get_feature_vector(
            {"user_id": user_id}, 
            return_type="pandas",
        )

        # Enrich inputs with user features
        inputs["user_city"] = user_features['user_city'].values[0]
        inputs["age"] = user_features['age'].values[0] 
        inputs["user_interests"] = user_features['user_interests'].values[0]
        
        return {
            "instances": [inputs]
        }
    
    def postprocess(self, outputs):
        # Return ordered ranking predictions
        return {
            "predictions": self.ranking_server.predict({"instances": outputs["predictions"]}),
        }


Overwriting querymodel_transformer.py


In [147]:
# Copy transformer file into Hopsworks File System
uploaded_file_path = dataset_api.upload(
    "querymodel_transformer.py", 
    "Models", 
    overwrite=True,
)

# Construct the path to the uploaded script
transformer_script_path = os.path.join(
    "/Projects", 
    project.name, 
    uploaded_file_path,
)

Uploading /home/nkama/masters_thesis_project/thesis/notebooks/querymodel_transformer.py: 0.000%|          | 0/…

In [148]:
from hsml.transformer import Transformer

query_model_deployment_name = "querydeployment"

# Define transformer
query_model_transformer=Transformer(
    script_file=transformer_script_path, 
    resources={"num_instances": 0},
)

# Deploy the query model
query_model_deployment = query_model.deploy(
    name=query_model_deployment_name,
    description="Deployment that generates query embeddings from user and event features using the query model",
    resources={"num_instances": 0},
    transformer=query_model_transformer,
)

Deployment created, explore it at https://c.app.hopsworks.ai:443/p/1220788/deployments/371731
Before making predictions, start the deployment by using `.start()`


In [150]:
# Start the deployment
query_model_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [183]:
weather_ranking_deployment.get_logs(component="predictor", tail=200)


Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1220788/deployments/371733

DeployableComponentLogs(instance_name: 'weatherrankingdeployment-predictor-00001-deployment-6bb688n8znf', date: datetime.datetime(2025, 5, 21, 23, 37, 16, 64328)) 
INFO:root:Loading component module...
INFO:root:[PredictorModel] Initializing predictor for model: weatherrankingdeployment
INFO:root:[HopsworksModel] Initializing for model: weatherrankingdeployment
INFO:root:Artifact path contents: ['predictor-weather_ranking_predictor.py', 'transformer-simple_weather_transformer.py']
INFO:root:Model path contents: ['weather_ranking_model.pkl']
... execution time: 1.603723 seconds
INFO:root:Starting KServe server...
2025-05-21 21:36:33.556 8 kserve INFO [model_server.py:register_model():363] Registering model: weatherrankingdeployment
2025-05-21 21:36:33.556 8 kserve INFO [model_server.py:start():298] Setting max asyncio worker threads as 12
2025-05-21 21:36:33.557 8 kserve I

In [182]:
weather_ranking_deployment.get_logs(component="transformer", tail=200)


Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1220788/deployments/371733

DeployableComponentLogs(instance_name: 'weatherrankingdeployment-transformer-00001-deployment-6896zq4tj', date: datetime.datetime(2025, 5, 21, 23, 37, 6, 848375)) 
INFO:root:Loading component module...
INFO:root:[TransformerModel] Initializing transformer for model: weatherrankingdeployment
INFO:root:[HopsworksModel] Initializing for model: weatherrankingdeployment
INFO:hsfs.engine.python:Python Engine initialized.
Downloading: 100.000%|██████████| 1094/1094 elapsed<00:00 remaining<00:00
INFO:transformer-simple_weather_transformer:Transformer initialized successfully
... execution time: 7.065520 seconds
INFO:root:Starting KServe server...
2025-05-21 21:35:37.328 8 kserve INFO [model_server.py:register_model():363] Registering model: weatherrankingdeployment
2025-05-21 21:35:37.328 8 kserve INFO [model_server.py:start():298] Setting max asyncio worker threads as 12
2025-05

In [ ]:
# Test ranking deployment
ranked_candidates = weather_ranking_deployment.predict(test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates, k=3)
recommendations